# Transformer-based Text Classfication

This notebook shows hot to make use of pre-trained Transformer models for building various types of text classifiers, some of which are multi-lingual. In particular, the notebook demonstrates the training of BERT-based text classifiers for: 
- categorising news group messages 
- detecting sentiment in Twitter messages
- determining if a sentence paraphrases another.

The notebook makes use of: 
- Transformer models from HuggingFace for training: https://github.com/huggingface/transformers
- Lime for explanations: https://lime-ml.readthedocs.io/en/latest/index.html

Much of the notebook is based on: 
- this blog post: https://towardsdatascience.com/text-classification-with-hugging-face-transformers-in-tensorflow-2-without-tears-ee50e4f3e7ed
- this Google Colab notebook: https://colab.research.google.com/drive/1YxcceZxsNlvK35pRURgbwvkgejXwFxUt
- this notebook: https://github.com/amaiya/ktrain/blob/master/examples/text/MRPC-BERT.ipynb


## Prepare environment

**NOTE**: To run this notebook, you will **need a GPU** (Graphics Processing Unit) card 
- the notebook has been tested on Google Colab and you are advised to run it there
- to use a GPU, you need to click on "Runtime" above, then "Change runtime type" and for "Hardware accelerator" choose the "GPU" option

Run the code below to check whether there is a gpu available:

In [ ]:
!pip3 install -q transformers datasets

In [ ]:
!pip install ktrain

In [ ]:
!pip3 uninstall flask
!pip3 install -q eli5

In [ ]:
import torch
import numpy as np

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type != 'cuda':
    raise SystemError('GPU device not found')

## Classification on the 20 Newsgroups dataset

In the first example we'll build a multi-class classifier for newsgroup messages, as examples of longer text messages.

Training on all of the documents in the dataset will take too long (a few hours), so we'll use just 4 today. 
- If you want to try on the full dataset, just remove the 'categories' attribute from the commands below. 
- Be warned though, that training time will be MUCH longer if you include all of the data, so first try subset of the categories.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'rec.sport.baseball', 'sci.med', 'alt.atheism']
train = fetch_20newsgroups(subset='train', shuffle=True, categories=categories)
test = fetch_20newsgroups(subset='test', shuffle=True, categories=categories)

Print out some basic information about the dataset:

In [ ]:
print('# train instances: ', len(train.data))
print('# test instances:  ', len(test.data))
print('classes: ', train.target_names)

Let's compute the occurrences of the classes

In [ ]:
from collections import Counter

counts = Counter(train.target)
counts

A class count shows that there is a relatively even distribution of messages across the categories:

In [ ]:
import matplotlib.pyplot as plt

plt.bar(counts.keys(), counts.values(), color="#3F5D7D", width=0.8);

### Model training and evaluation

#### Creating a model and preprocessing data

For the moment we will use the standard pretrained Multilingual BERT model. Note that this **model is big**! 
- it contains 110 million parameters
- it represents words (subword tokens to be precise) with embeddings of size 768 <-- compare this with the 50 dimensions we used with GloVe
- it contains 12 self-attention layers (each with 12 heads) stacked on top each other

In [ ]:
model_name = 'bert-base-multilingual-uncased'

There are many other pretrained models availalable here: https://huggingface.co/transformers/pretrained_models.html
- Give them a try and see which ones are faster and/or more stable to train!

Load the transformer setting the maximum text length to 500 and the classes to be the four newsgroup categories mentioned above.

In [ ]:
bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))

Move model to GPU

In [ ]:
bert = bert.to(device)

Now that we've downloaeded the base Multiligual BERT model, let's have a look at the architecture. 

In [ ]:
print(bert)

Check which of the parameters are trainable: 

In [ ]:
for param in bert.parameters():
    print(param.size(), param.requires_grad)

Load the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Now we can preprocess the training (and test) data using the transformer's data set API

First create a list of dictionaries with all the samples in the train and test data sets

In [ ]:
train_data = [{'text': txt, 'label': lbl} for txt, lbl in zip(train.data, train.target)]
test_data = [{'text': txt, 'label': lbl} for txt, lbl in zip(test.data, test.target)]

Convert the data sets into the Huggingface data set API format

In [ ]:
train_data = Dataset.from_list(train_data)
test_data = Dataset.from_list(test_data)

Create a container with all the splits of the data set

**Note**: We are using the same data for validation and testing, this is not a good practice, it's just for the tutoria, in general you should have separate data for validation and test.

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['validation'] = test_data
data['test'] = test_data

Finally use the tokenizer to convert the input strings into sequences of tokens (more on this later)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

Let's have a quick look at what the tokenised data looks like: 
- First print out the text of the first document:

In [ ]:
print(train.data[0])

Now print out the result of preprocessing the first document: 

In [ ]:
print(tokenizer(train.data[0]).input_ids)

Each integer above is the index of a token from the vocabulary of the BERT model. 
- How big is the vocabulary?

In [ ]:
print("vocabulary size: ", len(tokenizer.vocab))

This vocabulary is relatively small compared to many word embeddings discussed in the lecture, which often reach over 1 million distinct words. 
- The vocab size is kept relatively small through the use of sub-word tokens that are found using a byte-pair encoding scheme.
- We can take a single string and run it through the tokenizer to see the tokens produced as follows:

In [ ]:
example_text = "I always wondered what those Transformer models were doing to my text. Here is a OutOfVocabWord."
encoded_text = tokenizer(example_text).input_ids
vocab_terms = list(tokenizer.vocab.keys())
vocab_index = list(tokenizer.vocab.values())
print("input text: "+example_text)
print("tokenized:  ", [vocab_terms[vocab_index.index(i)] for i in encoded_text])

Note that:
- a '[CLS]' token has been added to the start of the tokenized text and a '[SEP]' token to the end.
- BERT works as a masked autoencoder, meaning that it is trained to produce the same terms as output that are presented in input, including those that were masked out.
- the '[CLS]' token is simply a special mask for the unknown class label that needs to be produced at the output. 
- the Byte-Pair Encoding has removed the suffixes from certain words (e.g. 'wondered' lost the 'ed'), but not all of them ('models' kept the 's'). 

#### Training the model

Now we're ready to start training the model. 

Prepare training arguments (including a name for the trainer)

In [ ]:
training_args = TrainingArguments(
    "cool_trainer_name", 
    per_device_train_batch_size=16
)

Disable weights in model encoder

In [ ]:
for param in bert.bert.parameters():
    param.requires_grad = False

Build the traininer using the Huggingface trainer API

In [ ]:
trainer = Trainer(
    model=bert, 
    args=training_args, 
    train_dataset=tokenized_data['train'], 
    eval_dataset=tokenized_data['validation']
)

Finally, run the training process (we train for three epochs)



In [ ]:
trainer.train()

Sorry Mark, no learning rate plot :(

#### Evaluating the model

Now that the model has finished training, let's evaluate in on the test data.

First we can run the evaluation process included with the trainer

In [ ]:
trainer.evaluate()

Then we can get the predictions on the test set...

In [ ]:
preds = trainer.predict(tokenized_data['test'])
preds

Convert predicted logits to classes using the $\mathrm{argmax}$ operator

In [ ]:
y_pred = torch.argmax(torch.tensor(preds.predictions), dim=1).numpy()
y_pred

Get target label names

In [ ]:
label_names = test.target_names

... and use them to preprare a classification report using Scikit-Learn API

In [ ]:
print(classification_report(test.target, y_pred, target_names=label_names))

... or compute a confusion matrix using the Scikit-Learn API

In [ ]:
print(confusion_matrix(test.target, y_pred))

We can have a look at the test examples on which the model made the worst predictions:

Compute loss

In [ ]:
loss = torch.nn.functional.cross_entropy(torch.tensor(preds.predictions), torch.tensor(preds.label_ids), reduction='none')
loss

Identify the index of the element with the highest loss

In [ ]:
index_of_instance = torch.argmax(loss).item()
index_of_instance

Loss of the corresponding sample

In [ ]:
loss[index_of_instance].item()

Predicted class and target class

In [ ]:
print(f'Target class:    {label_names[test.target[index_of_instance]]}')
print(f'Predicted class: {label_names[y_pred[index_of_instance]]}')

Print out the post with the top loss (by changing INDEX_OF_INSTANCE below) to try to find out why it is confusing the classfier. 

In [ ]:
print(test.data[index_of_instance])

### Making predictions on new data

We can run the trained classification model on new examples. 

#### Utility functions

Define a simple function to make a prediction

In [ ]:
@torch.no_grad()
def predict(text):
    input_encodings = tokenizer(text, return_tensors='pt').to(device)
    outputs = bert(**input_encodings)

    lbl = label_names[torch.argmax(outputs.logits).item()]

    return lbl

Define  simple function to predict the classes probabilities

In [ ]:
@torch.no_grad()
def predict_proba(text):
    input_encodings = tokenizer(text, return_tensors='pt').to(device)
    outputs = bert(**input_encodings)

    proba = torch.softmax(outputs.logits, dim=1).cpu().squeeze().numpy()

    
    return dict(zip(label_names, proba))

We can now try the trained model on new samples

In [ ]:
sample_text = 'Both of the home runs hit by Fernando Tatís in the third inning for the St. Louis Cardinals on April 23, 1999, were grand slams.'
predict(sample_text)

We can see the probabilities assigned by the model to each of the classes as follows:

In [ ]:
predict_proba(sample_text).values()

Where the classes are:

In [ ]:
predict_proba(sample_text).keys()

OK, so the model appears to work well on English text.

#### Multi-lingual examples

Let's try on a phrase in another language (Italian). 
- Note that the news group data we used to train the model is in English only, so the model has never seen any examples of news group messages in Italian.

In [ ]:
italian_text = 'I tasti su e giù sul mio portatile non funzionano più, quindi non posso più usarlo per giocare ;-('
predict(italian_text)

In [ ]:
predict_proba(italian_text)

Did it get the prediction right? 
- Note that while the characer set for Italian and English is almost the same, there weren't any words (except perhaps for "non") in the above message that would have occurred in the training data. 

Let's try a message in a different character set: 
- Translate the text into Chinese using Google Translate: https://translate.google.com/
- Then copy the text below and rerun the prediction. 
- Does it still work? 



In [ ]:
chinese_text = '笔记本电脑上的向上和向下键不再起作用，因此我无法再使用它来播放;-(' #'INSERT CHINESE TEXT HERE'
predict(chinese_text)

In [ ]:
russian_text = "Клавиши «вверх» и «вниз» на моем ноутбуке больше не работают, поэтому я больше не могу использовать их для игры ;-("
predict(russian_text)

### Explaining the predictions

Let's try another piece of text, this time in English, but talking about a politician who wasn't a politician (but rather a celebrity) at the time in which the 20 News Groups dataset was created:

In [ ]:
sample_text = 'Donald Trump is the greatest living ex-president in the history of living ex-presidents. He\'s also a bad golfer.'
predict(sample_text)

Let's invoke the `explain` method to see which words contribute most to the classification.

We will need to install the **eli5** library.

In [ ]:
import eli5
from eli5.lime import TextExplainer

Define funxtion to output the probability distribution as a matrix `(n_samples, n_classes)`

In [ ]:
@torch.no_grad()
def predict_proba_values(text):
    input_encodings = tokenizer(text, return_tensors='pt', padding=True).to(device)
    outputs = bert(**input_encodings)

    proba = torch.softmax(outputs.logits, dim=1).cpu().squeeze().numpy()

    return proba

Run the explainer (based on LIME) to see which words in the sentence are most important for the prediction:

In [ ]:
# See https://amaiya.github.io/ktrain/text/predictor.html
# See https://eli5.readthedocs.io/en/latest/autodocs/lime.html
def explain(text, truncate_len=512, all_targets=False, n_samples=2500):
    prediction = [predict(text)] if not all_targets else None
    text = " ".join(text.split()[:truncate_len])
    te = TextExplainer(random_state=42, n_samples=n_samples)
    _ = te.fit(text, predict_proba_values)
    return te.show_prediction(
        target_names=label_names, targets=prediction
    )

In [ ]:
explain(sample_text)

The words in the darkest shade of green contribute most to the classification.
- Do they agree with what you would have expected for this example?

### Inspecting the Model

Let's have a look at the architecture of the model that we have used:

In [ ]:
print(bert)

Compute the total number of parameters

In [ ]:
n_params = sum(param.numel() for param in bert.parameters())
n_params

The model contains over 100 million parameters consists of:
- BERT (which contains alost all the parameters), 
- a drop-out layer (that is inserted to prevent overfitting), and 
- a dense (feedforward) layer to map the BERT embedding of size 768 to the 2 output neurons (representing the positive and negative classes).

Let's print out information on the configuration of BERT.

In [ ]:
bert.bert.config

### Saving and loading a fine-tuned model

We can save predictor for later use

In [ ]:
bert.save_pretrained('/tmp/my_predictor')

Reload the predictor and use it to predict the class of a new piece of text:

In [ ]:
bert = AutoModelForSequenceClassification.from_pretrained('/tmp/my_predictor').to(device)
predict('My computer monitor is really blurry.')

## Classifying the sentiment of Twitter messages

We'll now train a different model to detect sentiment on Twitter data as we did for the previous tutorials:

In [ ]:
import nltk
nltk.download('twitter_samples')

from nltk.corpus import twitter_samples
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

import re 
emoticon_regex = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
positive_tweets_noemoticons = [re.sub(emoticon_regex,'',tweet) for tweet in positive_tweets]
negative_tweets_noemoticons = [re.sub(emoticon_regex,'',tweet) for tweet in negative_tweets]

tweets_x = positive_tweets_noemoticons + negative_tweets_noemoticons
tweets_y = ['positive']*len(positive_tweets) + ['negative']*len(negative_tweets)

Use scikit-learn to split the data into training, validation and test sets:


In [ ]:
from sklearn.model_selection import train_test_split
temp_x, test_x, temp_y, test_y = train_test_split(tweets_x, tweets_y, test_size=0.2)
train_x, valid_x, train_y, valid_y = train_test_split(temp_x, temp_y, test_size=0.2)

This time we are going to use a different tool called [ktrain](https://github.com/amaiya/ktrain): 

In [ ]:
import ktrain
from ktrain import text

1.   Load the model (this time we'll try DistilBERT, which is a smaller transformer model)

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'
transformer = text.Transformer(MODEL_NAME, maxlen=500, classes=['positive','negative'])

2. Process the training/test data

In [ ]:
processed_train = transformer.preprocess_train(train_x, train_y)
processed_test = transformer.preprocess_test(valid_x, valid_y)

3. Create a model and learner

In [ ]:
model = transformer.get_classifier()
learner = ktrain.get_learner(model, train_data=processed_train, val_data=processed_test, batch_size=6)

4.   Train the model (We'll run for one epoch to make it faster, but it would be better to run for 4 or more)

In [ ]:
learner.fit_onecycle(5e-5, 1)

5.   Evaluate the model

In [ ]:
learner.validate(class_names=transformer.get_classes())

6.   Make some predictions with it

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=transformer)
predictor.predict('Had a lot of fun today!')

Did it work?

Let's print out some information about the architecture of this model:

In [ ]:
print(model.summary())

We see that the distilbert model has fewer parameters than the BERT model we used previously. It's interesting to see the pre-classifier layer in this model that maps the output embeddings from the transformer into a second embedding space (of the same size) that is then used as features for the final classification layer.


## Pairwise classification task 

We now use the Microsoft Research Paraphrase Corpus (MRPC) to build a classifier for detecting paraphrased sentences. 
- This is an example of a paired sentence classfication task, where the prediction model takes in a **pair of texts as input** and produces a **binary label as output**.

We'll first use pandas (Python's data analysis library) to download and parse the tab-separated data:

In [ ]:
import pandas as pd
import csv
train_file = 'https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt'
test_file = 'https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt'
train_df = pd.read_csv(train_file, delimiter='\t', quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(test_file, delimiter='\t', quoting=csv.QUOTE_NONE)

We need to format the training data in the format needed for ktrain
- so select the two columns containing strings 
- and get their values (as an 2d array) 
- then convert the array to a list of tuples (str,str) as required by ktrain:

In [ ]:
x_train = train_df[['#1 String', '#2 String']].values
# for sentence pair classification, ktrain expects a list of tuples of form (str, str)
x_train = list(map(tuple, x_train))

Repeat for the test data:

In [ ]:
x_test = test_df[['#1 String', '#2 String']].values
x_test = list(map(tuple, x_test))

And of course, we need also the labels for the training / test examples:

In [ ]:
y_train = train_df['Quality'].values
y_test = test_df['Quality'].values

Print out sizes of dataset:


In [ ]:
print("# train instances: ",len(x_train))
print("# test instances:  ",len(x_train))

Have a look at a random sentence pair:

In [ ]:
import random 
i = random.randint(0,len(x_train)-1)
print("Sentance 1: "+x_train[i][0])
print("Sentence 2: "+x_train[i][1])
print("Label:      "+str(y_train[i]))

Build and train a model:

In [ ]:
MODEL_NAME = 'bert-base-uncased'
transformer = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not paraphrase', 'paraphrase'])
processed_train = transformer.preprocess_train(x_train, y_train)
processed_test = transformer.preprocess_test(x_test, y_test)
model = transformer.get_classifier()
learner = ktrain.get_learner(model, train_data=processed_train, val_data=processed_test, batch_size=32)
learner.fit_onecycle(5e-5, 3)

Get the predictor object:

In [ ]:
predictor = ktrain.get_predictor(learner.model, transformer)

And try the model on a pair of sentences:

In [ ]:
#predictor.predict(('Their chestnut cat stretched out its legs on the antique wooden desk in the centre of his study.','The cat was on the table.'))
predictor.predict(('The cat stretched out on the antique wooden desk in the study.','The cat was on the table.'))

Did it work? 
- Try it out on some of your own sentences!